# Regular Tracking + Taking Action = Positive Health Gains

### Setup

In [ ]:
import pathlib

In [ ]:
import pandas as pd

In [ ]:
date_partition = '20200922'

In [ ]:
home = pathlib.Path.home()

In [ ]:
data_input_path = f"{home}/small-data/apple-health-csv/full-extract/{date_partition}"

In [ ]:
study_path = f"{home}/small-data/study/health-stories/{date_partition}"

In [ ]:
pathlib.Path(study_path).mkdir(parents=True,exist_ok=True)

In [ ]:
bodymass = pd.read_csv(f"{data_input_path}/bodymass-summary.csv",parse_dates=['date'])

In [ ]:
body_weight = bodymass.rename(columns={'bodymass':'weight'})

In [ ]:
body_weight = body_weight.loc[body_weight['date']>'2018-04-01']

### Daily Perspective of Weight

In [ ]:
 daily_weight = body_weight.plot.line(title="Daily Weight, April 4, 2019 - September 22, 2020", 
                                      x='date', y='weight', 
                                      figsize=(23, 11), style='.-', 
                                      markevery=1, markersize=12, markerfacecolor='orange')

In [ ]:
fig = daily_weight.get_figure()

In [ ]:
fig.savefig(f"{study_path}/daily_weight.png")

### Monthly Statistics

In [ ]:
weights_by_month = body_weight.copy()
del weights_by_month['unit']

In [ ]:
weights_by_month['year_month'] = weights_by_month['date'].apply(lambda x: f"{x.year}-{x.month:02}")

In [ ]:
monthly_weigh_ins = weights_by_month.groupby(by="year_month").count()

In [ ]:
monthly_weigh_ins = monthly_weigh_ins.rename(columns={"weight": "weigh_ins"})

#### Monthly and Cumulative Weight Loss/Gain

In [ ]:
monthly_first_weighin = weights_by_month.groupby(['year_month'], as_index = False).first().loc[:, ("year_month", "weight")]

In [ ]:
monthly_first_weighin['monthly_change'] = monthly_first_weighin['weight'].diff(periods=1)

In [ ]:
monthly_first_weighin['cumulative_change'] = monthly_first_weighin['monthly_change'].cumsum()

In [ ]:
mfwi_plot = monthly_first_weighin.plot.bar(x='year_month', 
                                y=['monthly_change', 'cumulative_change'], 
                                title="Monthly Weight Loss/Gain (lbs) (First Weigh-in Each Month)",
                                width=0.90,
                                figsize = (23, 11), ylim=(-35, 15))
                               

In [ ]:
monthly_first_weighin

#### Monthly Averages

In [ ]:
monthly_average_weight = weights_by_month.groupby(by="year_month").mean()

In [ ]:
monthly_average_weight.columns

In [ ]:
monthly_average_weight.index

In [ ]:
# index is year_month, resetting the index results in year_month column
monthly_average_weight = monthly_average_weight.reset_index() 

In [ ]:
monthly_average_weight = monthly_average_weight.rename(columns={'weight': 'weight_lbs'})

In [ ]:
monthly_average_weight.columns

In [ ]:
maw_figure = monthly_average_weight.plot.line(title="Monthly Average Weight",
                                              figsize=(23, 11), style='.-', 
                                              markerfacecolor='orange',
                                              markersize=12)

In [ ]:
maw_figure.get_figure().savefig(f"{study_path}/monthly_average_weight.png")

##### Calculate monthly average weight changes

In [ ]:
monthly_average_weight['monthly_lbs_change'] = monthly_average_weight['weight_lbs'].diff(periods=1)

In [ ]:
monthly_average_weight['cumulative_lbs_change'] = monthly_average_weight['monthly_lbs_change'].cumsum()

In [ ]:
monthly_average_weight.columns

In [ ]:
mavg_deltas_plot = monthly_average_weight.plot.bar(
    x = "year_month", 
    y=['monthly_lbs_change', 'cumulative_lbs_change'], 
    title="Monthly Average Weight Loss/Gain (lbs)",
    width=0.90, figsize = (23, 11), ylim=(-35, 15))
                               

In [ ]:
monthly_average_weight

In [ ]:
monthly_weigh_ins = weights_by_month.groupby(by="year_month").count()

In [ ]:
monthly_weigh_ins = monthly_weigh_ins.reset_index()

In [ ]:
monthly_average_weight['weigh_ins'] = monthly_weigh_ins['weight']

In [ ]:
monthly_average_weight

In [ ]:
plot_monthly_avg_lb_changes_with_weighins = monthly_average_weight.plot.bar(
    title='Number of Monthly Weigh-ins and Weight Changes',
    x='year_month',
    y=['monthly_lbs_change', 'cumulative_lbs_change', 'weigh_ins', ], 
    figsize=(23, 11), width=0.9, ylim=(-35, 35))

In [ ]:
plot_monthly_avg_lb_changes_with_weighins.get_figure().savefig(f"{study_path}/monthly-avg-changes-with-weighins")

In [ ]:
daily_distance = pd.read_csv(f"{data_input_path}/distance-walking-running-summary.csv",parse_dates=['date'])

In [ ]:
del daily_distance['unit']

In [ ]:
daily_distance['year_month'] = daily_distance['date'].apply(lambda x : f"{x.year}-{x.month:02}")

In [ ]:
daily_distance

In [ ]:
import datetime as dt

In [ ]:
daily_average_distance = daily_distance.loc[(daily_distance['date'] < 
                                             dt.datetime(2020,9,22))].groupby("year_month").mean()

In [ ]:
daily_average_distance = daily_average_distance.rename(columns={'movement_distance': 'daily_average_miles'})

In [ ]:
monthly_stats = pd.merge(daily_average_distance, monthly_average_weight, 
                         left_on='year_month', right_on='year_month', how='left')

In [ ]:
monthly_stats

In [ ]:
plot_monthly_stats = monthly_stats.plot.bar(
    x='year_month',
    y=['monthly_lbs_change', 'cumulative_lbs_change', 'weigh_ins', 'daily_average_miles'],
    figsize=(23, 11), width=0.90, ylim=(-35, 35))

## Rolling Averages

In [ ]:
body_weight.dtypes

In [ ]:
del body_weight['unit']

In [ ]:
body_weight['rolling_mean'] = body_weight.rolling(7).mean()

In [ ]:
body_weight.dtypes

In [ ]:
body_weight['seven_day_weight_change'] = body_weight['weight'].diff(periods=7)

In [ ]:
body_weight.tail(120)

In [ ]:
body_weight = body_weight.loc[(body_weight['date'] > '2020-04-15') & (body_weight['date'] < '2020-06-16')]

In [ ]:
body_weight

In [ ]:
body_weight.plot.bar(title="Seven Day Weight Change", x='date', 
                      y=['seven_day_weight_change'], 
                      figsize=(23, 11))